## Incremental MSBWT Construction

In [6]:
def FMIndex(bwt):
    fm = [{c: 0 for c in bwt}]
    for c in bwt:
        row = {symbol: count + 1 if (symbol == c) else count for symbol, count in fm[-1].items()}
        fm.append(row)
    offset = {}
    N = 0
    for symbol in sorted(row.keys()):
        offset[symbol] = N
        N += row[symbol]
    return fm, offset

def inverseBWT(bwt):
    # initialize the table from t
    table = ['' for c in bwt]
    for j in range(len(bwt)):
        #insert the BWT as the first column
        table = sorted([c+table[i] for i, c in enumerate(bwt)])
    #return the row that ends with ‘$’
    return table[bwt.index('$')]

def BWT(t):
    # create a sorted list of all cyclic suffixes of t
    rotation = sorted([t[i:]+t[:i] for i in range(len(t))])
    # concatenate the last symbols from each suffix
    return ''.join(r[-1] for r in rotation)

def recoverSuffix(i, BWT, FMIndex, Offset):
    suffix = ''
    c = BWT[i]
    predec = Offset[c] + FMIndex[i][c]
    suffix = c + suffix
    while (predec != i):
        c = BWT[predec]
        predec = Offset[c] + FMIndex[predec][c]
        suffix = c + suffix
    return suffix

def findBWT(pattern, FMIndex, Offset):
    lo = 0
    hi = len(FMIndex) - 1
    for symbol in reversed(pattern):
        lo = Offset[symbol] + FMIndex[lo][symbol]
        hi = Offset[symbol] + FMIndex[hi][symbol]
    return lo, hi

In [7]:
bwt = "TGAG$TGC$AAA$AA"
fm, off = FMIndex(bwt)
for i in range(len(bwt)):
    print("%2d: %s" % (i, recoverSuffix(i, bwt, fm, off)))

new = "TATA$"
inserts = []
for i in range(len(new)):
    l, h = findBWT(new[i:]+new[:i],fm,off)
    inserts.append((h,new[i-1]))

for i, c in sorted(inserts, reverse=True):
    bwt = bwt[:i] + c + bwt[i:]

print(bwt)

 0: $ACAT
 1: $ATAG
 2: $GAGA
 3: A$GAG
 4: ACAT$
 5: AG$AT
 6: AGA$G
 7: AT$AC
 8: ATAG$
 9: CAT$A
10: G$ATA
11: GA$GA
12: GAGA$
13: T$ACA
14: TAG$A
TGAAGT$TGCT$AAA$AAA$


## msBWT merging in Python

In [7]:
def mergeBWT(bwt1, bwt2):
    interleave = [(c, 0) for c in bwt1] + [(c, 1) for c in bwt2]
    passes = min(len(bwt1), len(bwt2))
    for p in range(passes):
        i, j = 0, 0
        nextInterleave = []
        for c, k in sorted(interleave, key=lambda x: x[0]):
            if (k == 0):
                b = bwt1[i]
                i += 1
            else:
                b = bwt2[j]
                j += 1
            nextInterleave.append((b, k))
        if (nextInterleave == interleave):
            break
        interleave = nextInterleave
    return ''.join([c for c, k in interleave])

bwt1 = "TG$TC$AAAA"
bwt2 = "AAGTGTA$A$"
bwt12 = mergeBWT(bwt1, bwt2)
print(bwt12)
FM, Offset = FMIndex(bwt12)
# print(len(bwt12))
for i in range(len(bwt12)):
    j = (i>>2)+(i&3)*(len(bwt12)//4)
    print("%2d: %s" % (j, recoverSuffix(j,bwt12,FM,Offset)), "\n" if (i % 4 == 3) else "", end='')

TGAAGT$TGCT$AAA$AAA$
 0: $ACAT  5: A$TAT 10: ATA$T 15: GAGA$ 
 1: $ATAG  6: ACAT$ 11: ATAG$ 16: T$ACA 
 2: $GAGA  7: AG$AT 12: CAT$A 17: TA$TA 
 3: $TATA  8: AGA$G 13: G$ATA 18: TAG$A 
 4: A$GAG  9: AT$AC 14: GA$GA 19: TATA$ 
